In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [57]:
import json
import math
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def volume_of_cylinder(radius, height, unit="meter"):
    volume = math.pi * (radius * radius) * height
    cylinder_info = {
        "radius": radius,
        "height": height,
        "volume": volume,
        "unit": unit
    }
    return json.dumps(cylinder_info)

In [58]:
print(volume_of_cylinder(1,2))

{"radius": 1, "height": 2, "volume": 6.283185307179586, "unit": "meter"}


In [59]:
functions = [
    {
        "name": "volume_of_cylinder",
        "description": "Calculates the volume of a cylinder.",
        "parameters": {
            "type": "object",
            "properties": {
                "radius": {
                    "type": "integer",
                    "description": "Radius of the cylinder.",
                },
                "height": {
                    "type": "integer",
                    "description": "Height of the cylinder.",
                },
                "unit": {
                    "type": "string",
                    "enum": ["meter", "centimeter"],
                    "description": "Unit of measurement (meter or centimeter).",
                },
            },
            "required": ["radius", "height"],
        },
    }
]


In [71]:
radius = float(input("Enter the radius of the cylinder (in meters): "))
height = float(input("Enter the height of the cylinder (in meters): "))

messages = [
    {
        "role": "user",
        "content": f"Calculate the volume of a cylinder with radius {radius} meters and height {height} meters."
    }
]

Enter the radius of the cylinder (in meters): 1
Enter the height of the cylinder (in meters): 2


In [72]:
import openai

In [73]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,  # Use 'functions' for function calling
    function_call={"name": "volume_of_cylinder"}  # Allows automatic function calling
)


In [74]:
print(response)

{
  "id": "chatcmpl-BDmPk1jS3vXENlXkTGpDccPbrE3X0",
  "object": "chat.completion",
  "created": 1742624068,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "volume_of_cylinder",
          "arguments": "{\"radius\":1,\"height\":2,\"unit\":\"meter\"}"
        },
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 116,
    "completion_tokens": 14,
    "total_tokens": 130,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


In [75]:
response_message = response["choices"][0]["message"]

if "function_call" in response_message:
    function_call = response_message["function_call"]
    print("Function Call:", function_call)
else:
    print("No function call. Assistant responded with text:", response_message["content"])

Function Call: {
  "name": "volume_of_cylinder",
  "arguments": "{\"radius\":1,\"height\":2,\"unit\":\"meter\"}"
}


In [76]:
response_message

<OpenAIObject at 0x7f9a3b59a400> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "volume_of_cylinder",
    "arguments": "{\"radius\":1,\"height\":2,\"unit\":\"meter\"}"
  },
  "refusal": null,
  "annotations": []
}

In [77]:
response_message["content"]

In [78]:
response_message["function_call"]

<OpenAIObject at 0x7f9a3b6274a0> JSON: {
  "name": "volume_of_cylinder",
  "arguments": "{\"radius\":1,\"height\":2,\"unit\":\"meter\"}"
}

In [ ]:
args = json.loads(response["choices"][0]["message"].get("function_call", {}).get("arguments", "{}"))

# If API response doesn't provide values, ask the user for input
if not args:
    radius = float(input("Enter radius: "))
    height = float(input("Enter height: "))
    unit = input("Enter unit (default is meter): ") or "meter"
    args = {"radius": radius, "height": height, "unit": unit}

# Pass unpacked dictionary to the function
observation = volume_of_cylinder(**args)

print("Cylinder Volume Data:", observation)